<b><u>This notebook characterise peptide:lipid hydrogen bonds in AMP +  membrane systems</b></u>


Hbondanalysis module of mdAnalysis does not deal with lipid atoms by default. As a result, these must be entered
manually in the cell below. Otherwise the script should function as with other scripts. 


<i>To do: 

Write script to show the poppulations of hydrogen bonds Possible histogram.</l>

Change x axis of heatmap
</i>

In [13]:
import MDAnalysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import MDAnalysis.analysis.hbonds

In [14]:

##gromacs configuration file###
gro_file = 'fullsys.gro'
##compressed trajectory file###
trajectory_file = 'traj_comp.xtc'
u = MDAnalysis.Universe('%s'%gro_file, '%s'%trajectory_file)
##### Sim start and sim end time in ns
start = 0 
end = 200

##define lipid acceptors/donors. This has already been done for POPE and POPG 
lipid_acceptors  = ['031','032','021','O22','O11','O13','O14','OC3','OC2','N']
lipid_donors =['HO2','HO3','HN1','HN2','HN3']


In [15]:
################################# INSERT INFO FOR PEPTIDE1 ##########################################
peptide1 = dict()
#### Single letter sequence #####
peptide1['sequence'] = 'RKSKEKIGKEFKRIVQRIKDFLRNLVPRTES'
#### Peptide name #####
peptide1['peptide_name'] = 'RK-31'
#### Number of peptides #####
peptide1['pepnum'] = 4
### DO NOT EDIT ###
peptide1['resnum'] = len(peptide1['sequence'])
peptide1['restot'] = peptide1['pepnum']*peptide1['resnum']
peptide1['starting_resid'] = 1

In [16]:
#create dictionary of amino acids 
d = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
     'GLY': 'G', 'HSP': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}
#invert to permit single letter to be entered
inv_map = {v: k for k, v in d.items()}
#create function to generate 3 letter code with residue position
def single_three(sequence,resnum):
    single = (list(sequence)) 
    Aminos=[]
    for i in range(0,(resnum)):
        value = single[i]
        Aminos.append((str(i+1))+inv_map[value])
    return Aminos
peptide1['Aminos'] = single_three(peptide1['sequence'],peptide1['resnum'])
#Make list of peptides for plotting loops

In [17]:
#Run hydrogen bond analysis, get counts.
def hbond_heatmap_cal(peptide):
    total = peptide['Aminos']*peptide['pepnum']
    hbonds = []
    for i in range (peptide1['starting_resid'],(len(total)+peptide1['starting_resid'])):
        temp = MDAnalysis.analysis.hbonds.HydrogenBondAnalysis(u, "resid %s"%i, 
                                                               "resname POPE or resname POPG",update_selection="True",
                                                               distance=3.0,angle=120,donors=lipid_donors,
                                                               acceptors=lipid_acceptors)
        temp.run(step=100)
        count = temp.count_by_time()
        for j in range(0,len(count)):
            hbonds.append([count[j][0],total[(i-(peptide1['starting_resid']))],count[j][1]])

    df = pd.DataFrame(hbonds)    
    df.rename(columns={ df.columns[0]: "Time (ns)", df.columns[1]: "Residue",
                       df.columns[2]: "Hbond_num" }, inplace = True)
    df['Time (ns)'] = df['Time (ns)']/1000
    ##sum hydrogen bonds for each residue, unstack for wideform
    df = df.groupby(['Residue','Time (ns)'],sort=False).sum().unstack()
    df = df.reset_index()
    df['Section_Number'] = df['Residue'].str.replace('([A-Z]+)', '').astype(float)
    df = df.sort_values('Section_Number')
    df = df.drop('Section_Number',1)
    df = df.set_index('Residue')
    df.columns = df.columns.droplevel()
    return df
        
##create data frame, rearrange, plot as heatmap
def plot_hbond_heatmap(peptide):
    ## plot heatmap
    sns.set(font_scale=0.7)
    sns.heatmap(hbond_heatmap_cal(peptide),cbar_kws={'label': 'Number of hydrogen bonds'},
                xticklabels=25,yticklabels=1,cmap="Oranges")
    plt.xlabel('Time (ns)', fontsize=12)
    plt.ylabel('Residue', fontsize=12)
    plt.ylim(0,peptide['resnum'])
    #plt.xlim(start,end)
    plt.tick_params(
        which='both', 
        width = 0.6,
        bottom=True,
        left=True,
        labelbottom=True) 
    plt.savefig('hydrogen_bond_per_res.png',dpi=300,bbox_inches="tight")

In [ ]:
plot_hbond_heatmap(peptide1)

<b> This Cell will output a csv file with the sum of all hydrogen bonds. The output of this can be used in another notebook to make a boxplot </b>

<b> This Cell will output a csv file a running total of all hydrogen bonds to make an x,y plot </b>

In [18]:
def boxplot_hbond(peptide):
    output1=[]
    output2=[]
    for k in range(0,(peptide['pepnum'])):
        temp1 =[]
        timecount1 =[]
        for i in range(0,(peptide['resnum'])):
            timecount1.append(u.trajectory.time)
            number1 = (i+(k*peptide['resnum']))+peptide1['starting_resid']
            temp = MDAnalysis.analysis.hbonds.HydrogenBondAnalysis(u, "resid %s"%number1, 
                                                               "resname POPE or resname POPG",update_selection="True",
                                                               distance=3.0,angle=120,donors=lipid_donors,
                                                               acceptors=lipid_acceptors)
            temp.run(step=100)
            count = temp.count_by_time()
            value = pd.DataFrame(count)['count'].sum()
            output1.append([peptide['Aminos'][i],value,peptide['peptide_name']])
    df_out = pd.DataFrame(output1)
    return df_out

save_file = boxplot_hbond(peptide1)
save_file.to_csv('hbond_sum_%s.csv'%peptide1['peptide_name'])


1


/Users/philipferguson/.local/lib/python3.6/site-packages/MDAnalysis/analysis/hbonds/hbond_analysis.py:650: SelectionWarning: No donors found in selection 1. You might have to specify a custom 'donors' keyword. Selection will update so continuing with fingers crossed.
  warnings.warn(errmsg, category=SelectionWarning)
/Users/philipferguson/.local/lib/python3.6/site-packages/MDAnalysis/analysis/hbonds/hbond_analysis.py:650: SelectionWarning: No acceptors found in selection 1. You might have to specify a custom 'acceptors' keyword. Selection will update so continuing with fingers crossed.
  warnings.warn(errmsg, category=SelectionWarning)
/Users/philipferguson/.local/lib/python3.6/site-packages/MDAnalysis/analysis/hbonds/hbond_analysis.py:650: SelectionWarning: No acceptors found in selection 2. You might have to specify a custom 'acceptors' keyword. Selection will update so continuing with fingers crossed.
  warnings.warn(errmsg, category=SelectionWarning)
/Users/philipferguson/.local/li

2


KeyboardInterrupt: 

In [ ]:
boxplot_hbond(peptide1)

In [ ]:
#create selection
all_hbond_selection =MDAnalysis.analysis.hbonds.HydrogenBondAnalysis(u, "protein", 
                                                           "resname POPE or resname POPG",update_selection="True",
                                                            distance=3.0,angle=120,donors=lipid_donors,
                                                            acceptors=lipid_acceptors)
#run hbond analysis module
all_hbond_selection.run(step=100)
#create dataframe
df_all = pd.DataFrame(all_hbond_selection.count_by_time())
#save as csv
df_all.to_csv('hbond_count_%s.csv'%peptide1['peptide_name'], sep=',')

In [ ]:
###enter amino acid three letter code 
Amino_acid = "GLU"
amino_dict= d['%s'%Amino_acid]
number_of_aminos = sequence.count('%s'%amino_dict)

###set selection as the amino acid of interest. This could also be substited for resid ? 
selection =MDAnalysis.analysis.hbonds.HydrogenBondAnalysis(u, "resname %s"%Amino_acid, 
                                                           "resname POPE or resname POPG",update_selection="True",
                                                            distance=3.0,angle=120,donors=lipid_donors,
                                                            acceptors=lipid_acceptors)
#Change the start and step here, code below will alter occupancy calculation accordingly.
selection.run(step=1000,verbose=True)
selection.generate_table()


In [ ]:
##take the timeseries and put into a dataframe. group the dataframe and count each type of hydrogen bond
df_hbonds = pd.DataFrame.from_records(selection.table)
grouped = pd.DataFrame(df_hbonds.groupby(['donor_atom','acceptor_atom','donor_resnm','acceptor_resnm'],
                                  as_index = False).size()).reset_index()
grouped.rename(columns={ grouped.columns[4]: "count" }, inplace = True)

##multiple number of amino acids by time to work out how many rows correspond to frames
frames = number_of_aminos*16
##calculate occupancy, which is the number of frames the hydrogen bond is present
#In this instance occupancy is over the course of the whole simulation. 
grouped['occupancy (%)'] = (grouped['count']/frames)*100


In [ ]:
%%capture
#####SET UP POPE PLOTTING######

POPE = grouped[grouped.eq('POPE').any(axis=1)]
#The following is a really long series of steps to label backbone. Consider improving
POPE['Location1'] = POPE['donor_atom'].apply(lambda x: 1 if x == 'HN' else 0)

POPE['Location2'] = POPE['acceptor_atom'].apply(lambda x: 1 if x == 'N' else \
                                              ( 1 if x == 'O' else 0))
POPE['Location3']= POPE['Location1']+POPE['Location2']
POPE['Location'] = POPE['Location3'].apply(lambda x: 'Sidechain' if x < 1 else 'Backbone')
POPE.drop(POPE.columns[6:9], axis=1, inplace=True)
# combine column names to give donor-acceptor pairs 
POPE['Names'] = POPE['donor_atom'] + "(" + POPE['donor_resnm'] + ")" + "-" + \
POPE['acceptor_atom'] + "(" + POPE['acceptor_resnm'] + ")"

#####SET UP POPG PLOTTING######

POPG = grouped[grouped.eq('POPG').any(axis=1)]
#The following is a really long series of steps to label backbone. Consider improving
POPG['Location1'] = POPG['donor_atom'].apply(lambda x: 1 if x == 'HN' else 0)

POPG['Location2'] = POPG['acceptor_atom'].apply(lambda x: 1 if x == 'N' else \
                                              ( 1 if x == 'O' else 0))
POPG['Location3']= POPG['Location1']+POPG['Location2']
POPG['Location'] = POPG['Location3'].apply(lambda x: 'Sidechain' if x < 1 else 'Backbone')
POPG.drop(POPG.columns[6:9], axis=1, inplace=True)
# combine column names to give donor-acceptor pairs 
POPG['Names'] = POPG['donor_atom'] + "(" + POPG['donor_resnm'] + ")" + "-" + \
POPG['acceptor_atom'] + "(" + POPG['acceptor_resnm'] + ")"



In [ ]:
sns.set_style('whitegrid')
#prepare color labels
colors = {'Sidechain':'red','Backbone':'blue'}
POPE['color'] = POPE['Location'].apply(lambda x: colors[x])
#make figure
sns.barplot(x=POPE['Names'],y=POPE['occupancy (%)'],palette=POPE['color'])
sns.despine()
plt.xticks(rotation=90)

import matplotlib.patches as mpatches
red_patch = mpatches.Patch(color='red',label='Sidechain')
blue_patch = mpatches.Patch(color='Blue',label='Backbone')
plt.legend(handles=[red_patch,blue_patch])
plt.title('Hydrogen bonds with POPE')
plt.savefig('POPE_hbonds_%s.png'%Amino_acid,dpi=300,bbox_inches="tight")

In [ ]:
colors = {'Sidechain':'red','Backbone':'blue'}
POPG['color'] = POPG['Location'].apply(lambda x: colors[x])


sns.set_style('whitegrid')
sns.barplot(x=POPG['Names'],y=POPG['occupancy (%)'],palette=POPG['color'])
sns.despine()
plt.xticks(rotation=90)

import matplotlib.patches as mpatches
red_patch = mpatches.Patch(color='red',label='Sidechain')
blue_patch = mpatches.Patch(color='Blue',label='Backbone')
plt.legend(handles=[red_patch,blue_patch])
plt.title('Hydrogen bonds with POPG')
plt.savefig('POPG_hbonds_%s.png'%Amino_acid,dpi=300,bbox_inches="tight")